# Custom Project

This was largely unsuccessful as the best classifier's accuracy was only about 0.14, with chance being 0.1. I believe this may be due to lack of a good mask to use with the data. However I made my best effort at data loading and was partially successful in loading in, organizing, and preprocessing data and labels.

In [2]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from nilearn import plotting, image, input_data
from brainiak import image, io

In [3]:
data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'

In [14]:
! tree $'/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720/'

/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720/
├── CHANGES
├── dataset_description.json
├── README
├── sub-001
│   ├── anat
│   │   ├── sub-001_T1w.json
│   │   └── sub-001_T1w.nii -> ../../.git/annex/objects/jz/X6/MD5E-s54526304--ea40da7257f8d590f5ffd434f32dd5be.nii/MD5E-s54526304--ea40da7257f8d590f5ffd434f32dd5be.nii
│   └── func
│       ├── sub-001_task-Test_run-01_bold.json
│       ├── sub-001_task-Test_run-01_bold.nii -> ../../.git/annex/objects/VK/Xz/MD5E-s513884512--11926b3fc3f033f00b7c7ca1d3a74543.nii/MD5E-s513884512--11926b3fc3f033f00b7c7ca1d3a74543.nii
│       ├── sub-001_task-Test_run-01_events.csv
│       ├── sub-001_task-Test_run-01_events.tsv
│       ├── sub-001_task-Test_run-02_bold.json
│       ├── sub-001_task-Test_run-02_bold.nii -> ../../.git/annex/objects/M1/38/MD5E-s513884512--eb2d25460132691a59ef2c3eac8f653c.nii/MD5E-s513884512--eb2d25460132691a59ef2c3eac8f653c.nii
│       ├── sub-001_task-Test_run-02_events.csv
│       ├── sub-001_task

In [4]:
# aud_early (early auditory cortex)
aud_mask_file= '/gpfs/gibbs/project/cmhn/data/Sherlock_processed/masks/aud_early.nii'
aud_mask = io.load_boolean_mask(aud_mask_file)

# a1_rev (first auditory area)
a1_mask_file= '/gpfs/gibbs/project/cmhn/data/Sherlock_processed/masks/a1_rev.nii'
a1_mask = io.load_boolean_mask(a1_mask_file)

mask_names = ['aud_mask', 'a1_mask']
masks = [aud_mask, a1_mask]

In [5]:
num_subs = 5
subs = np.arange(num_subs)
numTRs = 410

In [6]:
def tsv_to_csv(s,t,r):
    tsv_file = data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.tsv'
    csv_table = pd.read_table(tsv_file,sep='\t')
    csv_table.to_csv(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.csv',index=False)

In [7]:
for s in subs:
    for t in ['Training','Test']:
        for r in [1,2,3,4,5,6]:
            tsv_to_csv(s+1,t,r)

In [8]:
def load_labels_for_str(s,t,r):
    df = pd.read_csv(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.csv') # pandas plays best with csv files
    return df

In [9]:
genres_for_str = load_labels_for_str(s,t,r)
genres_for_str['genre'].values

array(["'reggae'", "'blues'", "'pop'", "'classical'", "'disco'",
       "'metal'", "'jazz'", "'hiphop'", "'country'", "'rock'", "'reggae'",
       "'blues'", "'pop'", "'classical'", "'disco'", "'metal'", "'jazz'",
       "'hiphop'", "'country'", "'rock'", "'reggae'", "'blues'", "'pop'",
       "'classical'", "'disco'", "'metal'", "'jazz'", "'hiphop'",
       "'country'", "'rock'", "'reggae'", "'blues'", "'pop'",
       "'classical'", "'disco'", "'metal'", "'jazz'", "'hiphop'",
       "'country'", "'rock'", "'reggae'"], dtype=object)

In [10]:
def mask_preprocess_data(data, t, mask_name, mask, num_subs):
    """Mask and preprocess data for all subjects.

    Params:
    data: 4D fMRI data
    mask_name (str): name of mask
    mask: boolean mask
    num_subs: number of subjects

    Returns:
    mdataPath: path to all subjects' BOLD data for a mask
    """
    all_subject_data = []
    for subject in range(num_subs):
        masked_data = image.mask_image(data[subject], mask)
        scaler = preprocessing.StandardScaler().fit(masked_data)
        preprocessed_data = scaler.transform(masked_data)
        all_subject_data.append(preprocessed_data)

    mdataPath = os.path.join('/home', 'cmhn_ak2776', 'palmer_scratch', 'custom_music_final_project', f'{mask_name}_{t}_run{r}_data.npy')
    np.save(mdataPath, all_subject_data)
    print(f'Saved {mask_name} data for all subjects')
    return mdataPath

In [11]:
masks[0].shape

(61, 73, 61)

In [ ]:
def mask_preprocess_data(data, mask_name, mask, num_subs):
    """Mask and preprocess data for all subjects.

    Params:
    data: 4D fMRI data
    mask_name (str): name of mask
    mask: boolean mask
    num_subs: number of subjects

    Returns:
    mdataPath: path to all subjects' BOLD data for a mask
    """
    all_subject_data = []
    for subject in range(num_subs):
        masked_data = image.mask_image(data[subject], mask)
        scaler = preprocessing.StandardScaler().fit(masked_data)
        preprocessed_data = scaler.transform(masked_data)
        all_subject_data.append(preprocessed_data)

    mdataPath = os.path.join('/home', 'cmhn_ak2776', 'palmer_scratch', 'final_project', f'{mask_name}_data.npy')
    np.save(mdataPath, all_subject_data)
    print(f'Saved {mask_name} data for all subjects')
    return mdataPath

In [2]:
import pandas as pd

def loadlabels(sub, traintest, run):
    s = sub
    t = traintest
    r = run
    data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'
    genres = pd.read_table(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.tsv')
    return genres.iloc[:]['genre'].values

def loaddata(sub, traintest, run):
    s = sub
    t = traintest
    r = run
    data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'
    # bolddata = nib.load(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_bold.nii').get_fdata()
    bolddata = image.load_img(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_bold.nii')
    return bolddata

In [19]:
data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'
s = 1
t = 'Test'
r = 1
genres = pd.read_table(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.tsv')
genres

,onset,duration,genre,track,start,end
0,0,15,'pop',32,10.70,25.70
1,15,15,'rock',32,4.48,19.48
2,30,15,'country',85,1.07,16.07
3,45,15,'blues',17,6.44,21.44
4,60,15,'disco',75,11.03,26.03
5,75,15,'metal',39,13.11,28.11
6,90,15,'reggae',33,12.55,27.55
7,105,15,'classical',45,8.35,23.35
8,120,15,'hiphop',95,8.59,23.59
9,135,15,'jazz',44,9.49,24.49


In [5]:
genres.iloc[:]['genre'].values

array(["'pop'", "'rock'", "'country'", "'blues'", "'disco'", "'metal'",
       "'reggae'", "'classical'", "'hiphop'", "'jazz'", "'pop'", "'rock'",
       "'country'", "'blues'", "'disco'", "'metal'", "'reggae'",
       "'classical'", "'hiphop'", "'jazz'", "'pop'", "'rock'",
       "'country'", "'blues'", "'disco'", "'metal'", "'reggae'",
       "'classical'", "'hiphop'", "'jazz'", "'pop'", "'rock'",
       "'country'", "'blues'", "'disco'", "'metal'", "'reggae'",
       "'classical'", "'hiphop'", "'jazz'", "'pop'"], dtype=object)

In [21]:
from nilearn.input_data import NiftiMasker
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import sem
from copy import deepcopy



# print(f'Data shape - after masking: {maskedData.shape}')
labelstrain = np.repeat(loadlabels(1,'Training',1),10)
labelstest = np.repeat(loadlabels(1,'Test',1),10)
bolddatatrain = loaddata(1,'Training',1).get_fdata()
bolddatatest = loaddata(1,'Test',1).get_fdata()

NameError: name 'loadlabels' is not defined

In [20]:
# Create, fit, and apply normalization in one step.

# Assuming 'bolddata' is your 4D fMRI data array with shape (X, Y, Z, T)
X, Y, Z, T = bolddatatrain.shape

# Reshape the 4D array to a 2D array with dimensions (voxel, time)
bolddatatrain_2d = np.reshape(bolddatatrain, (X * Y * Z, T)).T
bolddatatest_2d = np.reshape(bolddatatest, (X * Y * Z, T)).T
print(1)
MaskBoldTrainNorm = preprocessing.StandardScaler().fit_transform(bolddatatrain_2d)
# Normalize the test data
MaskBoldTestNorm = preprocessing.StandardScaler().fit_transform(bolddatatest_2d)
print(2)
# 4. Run Classifier
np.random.seed(0)

# Pull out the sample data
train_data = MaskBoldTrainNorm
test_data = MaskBoldTestNorm
print(3)
#Train and test the classifier
classifier = SVC(kernel="linear", C=1)
clf = classifier.fit(train_data, labelstrain)
score = clf.score(test_data, labelstest)
print('Classification accuracy:', score)

NameError: name 'bolddatatrain' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(train_data, labelstrain)
best_classifier = grid.best_estimator_

# Test the classifier with the best hyperparameters
score = best_classifier.score(test_data, labelstest)
print('Classification accuracy:', score)

In [24]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from nilearn import plotting, image, input_data
from nilearn.input_data import NiftiMasker
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import sem
from copy import deepcopy


In [22]:
import pandas as pd

def loadlabels(sub, traintest, run):
    s = sub
    t = traintest
    r = run
    data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'
    genres = pd.read_table(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_events.tsv')
    return genres.iloc[:]['genre'].values

def loaddata(sub, traintest, run):
    s = sub
    t = traintest
    r = run
    data_dir ='/home/cmhn_ak2776/cmhn-s23-final-ajkohl/custom-project-resources/ds003720'
    #bolddata = nib.load(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_bold.nii').get_fdata()
    bolddata = image.load_img(data_dir + f'/sub-00{s}/func/sub-00{s}_task-{t}_run-{r:02d}_bold.nii')
    return bolddata

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# print(f'Data shape - after masking: {maskedData.shape}')
labelstrain = np.repeat(loadlabels(1,'Training',1),10)
labelstest = np.repeat(loadlabels(1,'Test',1),10)
bolddatatrain = loaddata(1,'Training',1)
bolddatatest = loaddata(1,'Test',1)

for roi in os.listdir('/gpfs/gibbs/project/cmhn/data/Pieman2/masks/rois/'):
    # 2. Load the mask file 
    mask = nib.load(f'/gpfs/gibbs/project/cmhn/data/Pieman2/masks/rois/{roi}')
    print(roi)
    # 3. Apply ROI mask
    nifti_masker = NiftiMasker(mask_img=mask)
    mbolddatatrain = nifti_masker.fit_transform(bolddatatrain)
    mbolddatatest = nifti_masker.fit_transform(bolddatatest)
    
    # Create, fit, and apply normalization in one step.
    MaskBoldTrainNorm = preprocessing.StandardScaler().fit_transform(mbolddatatrain)
    # Normalize the test data
    MaskBoldTestNorm = preprocessing.StandardScaler().fit_transform(mbolddatatest)
    
    # 4. Run Classifier
    np.random.seed(0)

    # Do voxel selection on all voxels
    #selected_voxels = SelectKBest(f_classif,k = MaskBoldTrainNorm.shape[1]//2).fit(MaskBoldTrainNorm, labelstrain)

    # Pull out the sample data
    train_data = MaskBoldTrainNorm
    test_data = MaskBoldTestNorm

    #Train and test the classifier
    classifier = SVC(kernel="linear", C=1)
    # clf = classifier.fit(selected_voxels.transform(train_data), labelstrain)
    # score = clf.score(selected_voxels.transform(test_data), labelstest)
    clf = classifier.fit(train_data, labelstrain)
    score = clf.score(test_data, labelstest)
    print('Classification accuracy:', score)

HG_R.nii
Classification accuracy: 0.0951219512195122
V1.nii
Classification accuracy: 0.0975609756097561
dMFG_R.nii
Classification accuracy: 0.12682926829268293
vPCUN.nii
Classification accuracy: 0.11463414634146342
vV2.nii
Classification accuracy: 0.09024390243902439
dPCUN.nii
Classification accuracy: 0.05609756097560976
aSTG_L.nii
Classification accuracy: 0.11707317073170732
dMFG_L.nii
Classification accuracy: 0.08780487804878048
STC_L.nii
Classification accuracy: 0.0951219512195122
PMC_L.nii
Classification accuracy: 0.12682926829268293
IPL_L.nii
Classification accuracy: 0.07317073170731707
hV4_R.nii
Classification accuracy: 0.1
SMA_L.nii
Classification accuracy: 0.0975609756097561
pMTS_L.nii
Classification accuracy: 0.1024390243902439
SFG.nii
Classification accuracy: 0.09024390243902439
aPFC.nii
Classification accuracy: 0.06341463414634146
dPCC.nii
Classification accuracy: 0.08536585365853659
cMTG_R.nii
Classification accuracy: 0.07804878048780488
dPoCS_L.nii
Classification accuracy:

In [18]:
from sklearn.svm import LinearSVC
from joblib import Parallel, delayed

import time

def process_roi(roi):
    start_time = time.time()
    # 2. Load the mask file 
    mask = nib.load(f'/gpfs/gibbs/project/cmhn/data/Pieman2/masks/rois/{roi}')
    print("Mask loading time:", time.time() - start_time)

    start_time = time.time()
    # 3. Apply ROI mask
    nifti_masker = NiftiMasker(mask_img=mask)
    mbolddatatrain = nifti_masker.fit_transform(bolddatatrain)
    mbolddatatest = nifti_masker.fit_transform(bolddatatest)
    print("ROI mask application time:", time.time() - start_time)

    # Create, fit, and apply normalization in one step.
    MaskBoldTrainNorm = preprocessing.StandardScaler().fit_transform(mbolddatatrain)
    # Normalize the test data
    MaskBoldTestNorm = preprocessing.StandardScaler().fit_transform(mbolddatatest)

    # 4. Run Classifier
    np.random.seed(0)
    clf_score = []

    # Do voxel selection on all voxels
    selected_voxels = SelectKBest(f_classif, k=MaskBoldTrainNorm.shape[1]//2).fit(MaskBoldTrainNorm, labelstrain)

    # Pull out the sample data
    train_data = MaskBoldTrainNorm
    test_data = MaskBoldTestNorm

    start_time = time.time()
    # Train and test the classifier
    classifier = LinearSVC(C=1)
    clf = classifier.fit(selected_voxels.transform(train_data), labelstrain)
    print("Classifier training time:", time.time() - start_time)

    start_time = time.time()
    score = clf.score(selected_voxels.transform(test_data), labelstest)
    print("Classifier scoring time:", time.time() - start_time)

    print('Classification accuracy:', score)
    return score

roi = roi_list[0]
process_roi(roi)


Mask loading time: 0.003696918487548828
ROI mask application time: 49.766361236572266
Classifier training time: 0.3711419105529785
Classifier scoring time: 0.0012061595916748047
Classification accuracy: 0.0975609756097561


0.0975609756097561

In [42]:
bolddatatrain.shape

(410, 174)

In [61]:
# Insert code here
np.random.seed(0)
clf_score = []

# Do voxel selection on all voxels
selected_voxels = SelectKBest(f_classif,k=400).fit(bolddatatrain, labelstrain)

# Pull out the sample data
train_data = bolddatatrain
test_data = bolddatatest

# Train and test the classifier
classifier = SVC(kernel="linear", C=1)
clf = classifier.fit(selected_voxels.transform(train_data), labelstrain)
score = clf.score(selected_voxels.transform(test_data), labelstest)
print('Classification accuracy:', score)

Classification accuracy: 0.1
